# Topic Modeling - Graphically

When considering topic modeling. It can be difficult to visualize the complexity when considering multiple topics. Presented is a means to create graphical concpetual model of documents in N-dimensions. 

For this exercise we will be using D3.js to produce our graphic for topic modeling dimensionality. To start using D3.js in JupyterLab or Google Colab, we need to include the following pre-run call. Meaning, each time we run a cell. We want JupyterLab to make the D3.js library available. 

In [1]:
from IPython.display import  HTML

def load_d3_in_cell_output():
  display(HTML("<script src='https://d3js.org/d3.v6.min.js'></script>"))
get_ipython().events.register('pre_run_cell', load_d3_in_cell_output)

## Topic Modeling -> Graphs

First, let's talk about what is actually happening when it comes to topic modeling and it translate to a 2D graphic. For this, we are going have three topics (a topic on 🐈, a topic on 🐦, and a topic on 🐳). We are predefining these for the sake of simplisty. 

Now that we have our topics, we need some documents to map to our topics. What we want to show is how related the document is to that given topic. If we are reading a document called "I love 🐦" we would expect it to map almost entirely to 🐦. Let's take a look at these extremes. 

```python
something = 10
something = something + 1

```

In [2]:
%%html
<div id="extremes"></div>

<script type="text/javascript">   
    var width = 600
    var height = 300
    var documents = []
    documents.push(({"title":"I love 🐦", "bird":1, "cat":0, "whale": 0, "icon":"🐦"}))
    documents.push(({"title":"Swimming with 🐳", "bird":0, "cat":0, "whale": 1, "icon":"🐳"}))
    documents.push(({"title":"Fancy 🐈", "bird":0, "cat":1, "whale": 0, "icon":"🐈"}))
    console.log(documents)
    var svg = d3.select("div#extremes").append("svg")
        .attr("width", width)
        .attr("height", height)
    
    var rectSize = 50
    var splits = 7
    svg.selectAll("rect.documents").data(documents)
        .join("rect")
        .attr("class","documents")
        .attr("x",(width/splits) - rectSize/2)
        .attr("y", (d,i)=>((height/5)*(i+1)) - rectSize/2)
        .attr("width", rectSize)
        .attr("height", rectSize)
        .style("fill","lightgrey")
        .style("stroke","black")
        .style("stroke-width",1)
        .on("mouseover", function(e,d){
            d3.select(e.currentTarget).transition().style("stroke-width",3)    
            var item = svg.selectAll("rect.documents").nodes();
            var i = item.indexOf(this);
            svg.select("circle#point_"+i)
                .transition()
                .duration(1000) 
                .ease(d3.easeBounce)
                .attr("cy",(height/5))
        })
        .on("mouseout", function(e,d){
            d3.select(e.currentTarget).transition().style("stroke-width",1)   
            var item = svg.selectAll("rect.documents").nodes();
            var i = item.indexOf(this);
            svg.select("circle#point_"+i)
                .transition()
                .attr("cy",(height/5)*4)
        })

    svg.selectAll("text.documents").data(documents)
        .join("text")
        .attr("class","documents")
        .attr("x",(width/splits) + (rectSize/2) + 10)
        .attr("y", (d,i)=>(height/5)*(i+1))
        .text(d=>d.title)
        
    svg.selectAll("line").data(documents)
        .join("line")
        .attr("x1", (d,i)=>(width/splits)*(i+4))
        .attr("y1", (height/5))
        .attr("x2", (d,i)=>(width/splits)*(i+4))
        .attr("y2", (height/5)*4)
        .style("stroke","black")
        .style("stroke-width",1)
        
    svg.selectAll("circle.topics").data(documents)
        .join("circle")
        .attr("class","topics")
        .attr("id", (d,i)=> "point_"+i)
        .attr("cx", (d,i)=>(width/splits)*(i+4))
        .attr("cy", (height/5)*4)
        .attr("r",5)
        .style("stroke","black")
        .style("stroke-width",1)
        
    svg.selectAll("text.topics").data(documents)
        .join("text")
        .attr("class","topics")
        .attr("x", (d,i)=>(width/splits)*(i+4))
        .attr("y", (height/5))
        .style("text-anchor", "middle")
        .text(d=>d.icon)
        
        
    svg.append("text")
        .attr("x",(width/splits))
        .attr("y",10)
        
        .text("Documents")

    svg.append("text")
        .attr("x",(width/splits)*5)
        .attr("y",10)
        .style("text-anchor", "middle")
        .text("Topics")
        
        
</script>

Based on this, when we have clearly defined documents, it is pretty easy to map them to a specific topic. But what's the fun in that? What about situations where documents have multiple topics. Let's take a look at this when our documents stretch over multiple topics. 

In [3]:
%%html
<div id="cotopic"></div>

<script type="text/javascript">   
    var width = 600
    var height = 300
    var topics = ["bird","cat","whale"]
    var documents = []
    documents.push(({"title":"The Adventures of 🐈 and 🐦", "bird":.5, "cat":.5, "whale": 0, "icon":"🐦"}))
    documents.push(({"title":"Mammels: 🐈 to 🐳", "bird":0, "cat":.5, "whale": .5, "icon":"🐳"}))
    documents.push(({"title":"At the Zoo 🐈, 🐳, and 🐦", "bird":.333, "cat":.333, "whale": .333, "icon":"🐈"}))
    console.log(documents)
    var svg = d3.select("div#cotopic").append("svg")
        .attr("width", width)
        .attr("height", height)
    
    var rectSize = 50
    var splits = 7
    svg.selectAll("rect.documents").data(documents)
        .join("rect")
        .attr("class","documents")
        .attr("x",(width/splits) - rectSize/2)
        .attr("y", (d,i)=>((height/5)*(i+1)) - rectSize/2)
        .attr("width", rectSize)
        .attr("height", rectSize)
        .style("fill","lightgrey")
        .style("stroke","black")
        .style("stroke-width",1)
        .on("mouseover", function(e,d){
            d3.select(e.currentTarget).transition().style("stroke-width",3)    
            var item = svg.selectAll("rect.documents").nodes();
            var i = item.indexOf(this);
            topics.forEach(function(e,j){
                var top = (height/5)
                var bottom = ((height/5)*4)
                var middle = bottom-top
                svg.select("circle#point2_"+j)
                    .transition()
                    .duration(1000) 
                    .ease(d3.easeBounce)
                    .attr("cy", bottom-(middle * documents[i][topics[j]]))  
            })

        })
        .on("mouseout", function(e,d){
            d3.select(e.currentTarget).transition().style("stroke-width",1)   
            var item = svg.selectAll("rect.documents").nodes();
            var i = item.indexOf(this);
            svg.selectAll("circle.point2")
                .transition()
                .attr("cy",(height/5)*4)
        })

    svg.selectAll("text.documents").data(documents)
        .join("text")
        .attr("class","documents")
        .attr("x",(width/splits) + (rectSize/2) + 10)
        .attr("y", (d,i)=>(height/5)*(i+1))
        .text(d=>d.title)
        
    svg.selectAll("line").data(documents)
        .join("line")
        .attr("x1", (d,i)=>(width/splits)*(i+4))
        .attr("y1", (height/5))
        .attr("x2", (d,i)=>(width/splits)*(i+4))
        .attr("y2", (height/5)*4)
        .style("stroke","black")
        .style("stroke-width",1)
        
    svg.selectAll("circle.point2").data(documents)
        .join("circle")
        .attr("class","point2")
        .attr("id", (d,i)=> "point2_"+i)
        .attr("cx", (d,i)=>(width/splits)*(i+4))
        .attr("cy", (height/5)*4)
        .attr("r",5)
        .style("stroke","black")
        .style("stroke-width",1)
        
    svg.selectAll("text.topics").data(documents)
        .join("text")
        .attr("class","topics")
        .attr("x", (d,i)=>(width/splits)*(i+4))
        .attr("y", (height/5))
        .style("text-anchor", "middle")
        .text(d=>d.icon)
        
        
    svg.append("text")
        .attr("x",(width/splits))
        .attr("y",10)
        
        .text("Documents")

    svg.append("text")
        .attr("x",(width/splits)*5)
        .attr("y",10)
        .style("text-anchor", "middle")
        .text("Topics")
        
        
</script>

In [4]:
%%html
<input type="range" min="1" max="10" value="0" name="topics" oninput="graph(+this.value)">
<label for="topics">Weight Strength: </label><em id="topics" style="font-style: normal;">0</em>
    
<input type="button" onclick="addDocument()" value="Add More Documents">
<div id="forceTopics"></div>

<script type="text/javascript"> 
    var simulation
    var width = 400
    var height = 400
    var margin = 30
    var numOfDocs = 1
    var currentWeighting = 0
    var tri_x = [(width/2), width-margin, margin]
    var margin = 60
    var tri_y = [margin, height-margin, height-margin]   
    nodes = []
    nodes.push(({"name":"🐦","fx":tri_x[0],"fy":tri_y[0]}))
    nodes.push(({"name":"🐳","fx":tri_x[1],"fy":tri_y[1]}))
    nodes.push(({"name":"🐈","fx":tri_x[2],"fy":tri_y[2]}))
    nodes.push(({"name":"🐈 and 🐦", "bird":.5, "cat":.5, "whale": 0, "icon":"🐦"}))
    nodes.push(({"name":"🐈 to 🐳", "bird":0, "cat":.5, "whale": .5, "icon":"🐳"}))
    nodes.push(({"name":"🐈, 🐳, and 🐦", "bird":.333, "cat":.333, "whale": .333, "icon":"🐈"}))
    links = []  
    links.push(({"source":nodes[0],"target":nodes[3],"weight":.5}))
    links.push(({"source":nodes[1],"target":nodes[3],"weight":.5}))
    links.push(({"source":nodes[2],"target":nodes[3],"weight":0}))
    links.push(({"source":nodes[0],"target":nodes[4],"weight":0}))
    links.push(({"source":nodes[1],"target":nodes[4],"weight":.5}))
    links.push(({"source":nodes[2],"target":nodes[4],"weight":.5}))
    links.push(({"source":nodes[0],"target":nodes[5],"weight":.333}))
    links.push(({"source":nodes[1],"target":nodes[5],"weight":.333}))
    links.push(({"source":nodes[2],"target":nodes[5],"weight":.333}))
    var svg = d3.select("div#forceTopics").append("svg")
        .attr("width", width)
        .attr("height", height)          
              
    var simulation = d3.forceSimulation(nodes)
        .force("link", d3.forceLink(links).id(d => d.name).strength(d=>d.weight*(0/10)))
        .force("charge", d3.forceManyBody().strength(-30))
        .force("center", d3.forceCenter(width / 2, height / 2))
        .on("tick", ticked)
  

    var link = svg.selectAll("line.links")
        .data(links).join("line")
        .attr("class","links")
        .style("stroke", "lightgrey")
        .style("stroke-opacity", 0.6)
        .attr("stroke-width", d => Math.sqrt(10*d.weight))

    var scale = d3.scaleOrdinal(d3.schemeCategory10);
    var node = svg.selectAll("text.nodes")
        .data(nodes).join("text")
        .attr("class","nodes")
        .style("font-size", "20px")
        .style("text-anchor", "middle")
        .text(d => d.name)

    node.append("title")
      .text(d => d.name);

    function ticked() {
        svg.selectAll("text.nodes")
            .attr("x",d=>d.x)
            .attr("y",d=>d.y)
        
        svg.selectAll("line.links")
            .attr("x1", d => d.source.x)
            .attr("y1", d => d.source.y)
            .attr("x2", d => d.target.x)
            .attr("y2", d => d.target.y)
        }
    
function graph(n) {

    currentWeighting = n
    var width = 400
    var height = 400
    simulation
    .force("link", d3.forceLink(links).id(d => d.name).strength(d=>d.weight*(+n/10)))
    .force("center", d3.forceCenter(width / 2, height / 2).strength(1-(+n/10)))
    .alpha(.5)
    .alphaTarget(0.3)
    .restart();
}
function addDocument() {
    
    nodes.push(({"name":"Doc_"+numOfDocs}))
    simulation.nodes(nodes)
    w1 = Math.round(Math.random()*10)
    w2 = Math.floor(Math.random() * (10 - w1))
    w3 = (10 - w1) - w2

    links.push(({"source":nodes[0],"target":nodes[5+numOfDocs],"weight":w1/10}))
    links.push(({"source":nodes[1],"target":nodes[5+numOfDocs],"weight":w2/10}))
    links.push(({"source":nodes[2],"target":nodes[5+numOfDocs],"weight":w3/10}))
    simulation.force("link", d3.forceLink(links).id(d => d.name).strength(d=>d.weight*(currentWeighting/10)))

    numOfDocs++
    updateGraph()
    

}
function updateGraph() {
    svg.selectAll("line.links")
        .data(links).join("line")
        .attr("class","links")
        .style("stroke", "#999")
        .style("stroke-opacity", 0.6)
        .attr("stroke-width", d => Math.sqrt(10*d.weight))
    svg.selectAll("text")
        .data(nodes).join("text")
        .attr("class","nodes")
        .style("font-size", "20px")
        .style("text-anchor", "middle")
        .text(d => d.name)    

}


</script>

Topics = 5

$n = 5$

$r = 3$

$\frac{n!}{ r! (n - r)!}$

In [5]:
%%html
<div id="emoji1"></div>

<script type="text/javascript">   
    var width = 500
    var height = 200
    var margin = 30
    var fac = n => !(n > 1) ? 1 : fac(n - 1) * n;
    var n = 5
    var r = 3 
    var soup = '🐈,🐦,🐳,🐧,🐕,🐙,🐝,🐄,🐪,🐍,🐞,🐬,🐑,🐉,🐤,🐢,🐒,🐘,🐠,🐁'.split(',');
    
    var topics = d3.range(n).map(d=>soup[d])
    var sets = []
    for (let i = 0; i < topics.length - 1; i++) {
        for (let j = i+1; j < topics.length - 1; j++) {
            for (let k = j+1; k < topics.length; k++) {
                var temp = []
                temp.push(topics[i])
                temp.push(topics[j])
                temp.push(topics[k])
                sets.push(temp)
            }
        }
    }

    var triangles = fac(n) / ( fac(r) * fac(n - r) )
    var tri_col = 10
    var tri_row = Math.ceil(sets.length/tri_col)
    var tri_size = 20
    var x = d3.scaleLinear().range([margin , width - margin]).domain([0,tri_col-1])
    var y = d3.scaleBand().range([margin, height-margin]).domain(d3.range(tri_row))
  
    var tri_x = [(tri_size/2), tri_size, 0]
    var tri_y = [0, tri_size, tri_size]   
    var palette = d3.interpolateTurbo
    var color = d3.scaleLinear().range([0,1]).domain([0,topics.length-1])
    
    var svg = d3.select("div#emoji1").append("svg")
        .attr("width", width)
        .attr("height", height)
        
    var g = svg.selectAll("g")
        .data(sets)
        .join("g")
        .attr("transform", (d,i) => "translate("+(x(i%tri_col))+","+(y(Math.floor(i/tri_col))+(y.bandwidth()/2))+")")
    g.append("line")
        .attr("x1", (d,i)=> tri_x[0])
        .attr("y1", (d,i)=> tri_y[0])
        .attr("x2", (d,i)=> tri_x[1])
        .attr("y2", (d,i)=> tri_y[1])
        .style("stroke", "black")
        .style("stroke-width", "1px")
    g.append("line")
        .attr("x1", (d,i)=> tri_x[1])
        .attr("y1", (d,i)=> tri_y[1])
        .attr("x2", (d,i)=> tri_x[2])
        .attr("y2", (d,i)=> tri_y[2])
        .style("stroke", "black")
        .style("stroke-width", "1px")
    g.append("line")
        .attr("x1", (d,i)=> tri_x[2])
        .attr("y1", (d,i)=> tri_y[2])
        .attr("x2", (d,i)=> tri_x[0])
        .attr("y2", (d,i)=> tri_y[0])
        .style("stroke", "black")
        .style("stroke-width", "1px")    
    
    g.selectAll("text")
        .data(d=>d)
        .join("text")
        .attr("x", (d,i)=> tri_x[i])
        .attr("y", (d,i)=> tri_y[i])
        .style("text-anchor", "middle")
        .style("alignment-baseline","middle")
        .text(d=>d)

</script>

Animals = 6 

In [6]:
%%html
<div id="emoji2"></div>

<script type="text/javascript">   
    var width = 600
    var height = 600
    var margin = 30
    var fac = n => !(n > 1) ? 1 : fac(n - 1) * n;
    var n = 6
    var r = 3 
    var soup = '🐈,🐦,🐳,🐧,🐕,🐙,🐝,🐄,🐪,🐍,🐞,🐬,🐑,🐉,🐤,🐢,🐒,🐘,🐠,🐁'.split(',');
    
    var topics = d3.range(n).map(d=>soup[d])
    var sets = []
    for (let i = 0; i < topics.length - 1; i++) {
        for (let j = i+1; j < topics.length - 1; j++) {
            for (let k = j+1; k < topics.length; k++) {
                var temp = []
                temp.push(topics[i])
                temp.push(topics[j])
                temp.push(topics[k])
                sets.push(temp)
            }
        }
    }

    var triangles = fac(n) / ( fac(r) * fac(n - r) )
    var tri_col = 10
    var tri_row = Math.ceil(sets.length/tri_col)
    var tri_size = 20
    var x = d3.scaleLinear().range([margin , width - margin]).domain([0,tri_col-1])
    var y = d3.scaleBand().range([margin, height-margin]).domain(d3.range(tri_row))
    console.log(d3.range(tri_row))
  
    var tri_x = [(tri_size/2), tri_size, 0]
    var tri_y = [0, tri_size, tri_size]   
    var palette = d3.interpolateTurbo
    var color = d3.scaleLinear().range([0,1]).domain([0,topics.length-1])
    console.log(y.bandwidth())
    
    var svg = d3.select("div#emoji2").append("svg")
        .attr("width", width)
        .attr("height", height)
        
    var g = svg.selectAll("g")
        .data(sets)
        .join("g")
        .attr("transform", (d,i) => "translate("+(x(i%tri_col))+","+(y(Math.floor(i/tri_col))+(y.bandwidth()/2))+")")
    g.append("line")
        .attr("x1", (d,i)=> tri_x[0])
        .attr("y1", (d,i)=> tri_y[0])
        .attr("x2", (d,i)=> tri_x[1])
        .attr("y2", (d,i)=> tri_y[1])
        .style("stroke", "black")
        .style("stroke-width", "1px")
    g.append("line")
        .attr("x1", (d,i)=> tri_x[1])
        .attr("y1", (d,i)=> tri_y[1])
        .attr("x2", (d,i)=> tri_x[2])
        .attr("y2", (d,i)=> tri_y[2])
        .style("stroke", "black")
        .style("stroke-width", "1px")
    g.append("line")
        .attr("x1", (d,i)=> tri_x[2])
        .attr("y1", (d,i)=> tri_y[2])
        .attr("x2", (d,i)=> tri_x[0])
        .attr("y2", (d,i)=> tri_y[0])
        .style("stroke", "black")
        .style("stroke-width", "1px")    
    
    g.selectAll("text")
        .data(d=>d)
        .join("text")
        .attr("x", (d,i)=> tri_x[i])
        .attr("y", (d,i)=> tri_y[i])
        .style("text-anchor", "middle")
        .text(d=>d)

</script>

Animals = 7

In [7]:
%%html
<div id="emoji3"></div>

<script type="text/javascript">   
    var width = 600
    var height = 600
    var margin = 30
    var fac = n => !(n > 1) ? 1 : fac(n - 1) * n;
    var n = 7
    var r = 3 
    var soup = '🐈,🐦,🐳,🐧,🐕,🐙,🐝,🐄,🐪,🐍,🐞,🐬,🐑,🐉,🐤,🐢,🐒,🐘,🐠,🐁'.split(',');
    
    var topics = d3.range(n).map(d=>soup[d])
    var sets = []
    for (let i = 0; i < topics.length - 1; i++) {
        for (let j = i+1; j < topics.length - 1; j++) {
            for (let k = j+1; k < topics.length; k++) {
                var temp = []
                temp.push(topics[i])
                temp.push(topics[j])
                temp.push(topics[k])
                sets.push(temp)
            }
        }
    }

    var triangles = fac(n) / ( fac(r) * fac(n - r) )
    var tri_col = 10
    var tri_row = Math.ceil(sets.length/tri_col)
    var tri_size = 20
    var x = d3.scaleLinear().range([margin , width - margin]).domain([0,tri_col-1])
    var y = d3.scaleBand().range([margin, height-margin]).domain(d3.range(tri_row))
    console.log(d3.range(tri_row))
  
    var tri_x = [(tri_size/2), tri_size, 0]
    var tri_y = [0, tri_size, tri_size]   
    var palette = d3.interpolateTurbo
    var color = d3.scaleLinear().range([0,1]).domain([0,topics.length-1])
    console.log(y.bandwidth())
    
    var svg = d3.select("div#emoji3").append("svg")
        .attr("width", width)
        .attr("height", height)
        
    var g = svg.selectAll("g")
        .data(sets)
        .join("g")
        .attr("transform", (d,i) => "translate("+(x(i%tri_col))+","+(y(Math.floor(i/tri_col))+(y.bandwidth()/2))+")")
    g.append("line")
        .attr("x1", (d,i)=> tri_x[0])
        .attr("y1", (d,i)=> tri_y[0])
        .attr("x2", (d,i)=> tri_x[1])
        .attr("y2", (d,i)=> tri_y[1])
        .style("stroke", "black")
        .style("stroke-width", "2px")
    g.append("line")
        .attr("x1", (d,i)=> tri_x[1])
        .attr("y1", (d,i)=> tri_y[1])
        .attr("x2", (d,i)=> tri_x[2])
        .attr("y2", (d,i)=> tri_y[2])
        .style("stroke", "black")
        .style("stroke-width", "2px")
    g.append("line")
        .attr("x1", (d,i)=> tri_x[2])
        .attr("y1", (d,i)=> tri_y[2])
        .attr("x2", (d,i)=> tri_x[0])
        .attr("y2", (d,i)=> tri_y[0])
        .style("stroke", "black")
        .style("stroke-width", "2px")    
    
    g.selectAll("text")
        .data(d=>d)
        .join("text")
        .attr("x", (d,i)=> tri_x[i])
        .attr("y", (d,i)=> tri_y[i])
        .style("text-anchor", "middle")
        .style("alignment-baseline","middle")
        .text(d=>d)

</script>

Topics = 8

You can see how the complexity increase with the number of topics. Using the slider, change the number of topics to see how much this space increase with the number of topics. 

In [8]:
%%html
<input type="range" min="3" max="15" value="3" name="topics" oninput="graph(+this.value)">
<label for="topics">Topics: </label><em id="topics" style="font-style: normal;">3</em>
<div id="emojiN"></div>

<script type="text/javascript">   
function graph(n) {
    document.getElementById('topics').innerHTML = n
    var width = 600
    var height = 2000
    var margin = 30
    var fac = n => !(n > 1) ? 1 : fac(n - 1) * n;
    var r = 3 
    var soup = '🐈,🐦,🐳,🐧,🐕,🐙,🐝,🐄,🐪,🐍,🐞,🐬,🐑,🐉,🐤,🐢,🐒,🐘,🐠,🐁'.split(',');
    
    var topics = d3.range(n).map(d=>soup[d])
    var sets = []
    for (let i = 0; i < topics.length - 1; i++) {
        for (let j = i+1; j < topics.length - 1; j++) {
            for (let k = j+1; k < topics.length; k++) {
                var temp = []
                temp.push(topics[i])
                temp.push(topics[j])
                temp.push(topics[k])
                sets.push(temp)
            }
        }
    }

    var triangles = fac(n) / ( fac(r) * fac(n - r) )
    var tri_col = 10
    var tri_row = Math.ceil(sets.length/tri_col)
    var tri_size = 20
    var x = d3.scaleLinear().range([margin , width - margin]).domain([0,tri_col-1])
    var y = d3.scaleBand().range([margin, height-margin]).domain(d3.range(tri_row))
  
    var tri_x = [(tri_size/2), tri_size, 0]
    var tri_y = [0, tri_size, tri_size]   
    var palette = d3.interpolateTurbo
    var color = d3.scaleLinear().range([0,1]).domain([0,topics.length-1])
    d3.select("div#emojiN").select("svg").remove()
    var svg = d3.select("div#emojiN").append("svg")
        .attr("width", width)
        .attr("height", height)
        
    var g = svg.selectAll("g")
        .data(sets)
        .join("g")
        .attr("transform", (d,i) => "translate("+(x(i%tri_col))+","+(y(Math.floor(i/tri_col))+(y.bandwidth()/2))+")")
    g.append("line")
        .attr("x1", (d,i)=> tri_x[0])
        .attr("y1", (d,i)=> tri_y[0])
        .attr("x2", (d,i)=> tri_x[1])
        .attr("y2", (d,i)=> tri_y[1])
        .style("stroke", "black")
        .style("stroke-width", "2px")
    g.append("line")
        .attr("x1", (d,i)=> tri_x[1])
        .attr("y1", (d,i)=> tri_y[1])
        .attr("x2", (d,i)=> tri_x[2])
        .attr("y2", (d,i)=> tri_y[2])
        .style("stroke", "black")
        .style("stroke-width", "2px")
    g.append("line")
        .attr("x1", (d,i)=> tri_x[2])
        .attr("y1", (d,i)=> tri_y[2])
        .attr("x2", (d,i)=> tri_x[0])
        .attr("y2", (d,i)=> tri_y[0])
        .style("stroke", "black")
        .style("stroke-width", "2px")    
    
    g.selectAll("text")
        .data(d=>d)
        .join("text")
        .attr("x", (d,i)=> tri_x[i])
        .attr("y", (d,i)=> tri_y[i])
        .style("text-anchor", "middle")
        .style("alignment-baseline","middle")
        .text(d=>d)

}
graph(3)
</script>